# Animation example

In [1]:
#@title Mount Google Drive
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    outputs_path = "/content/gdrive/MyDrive/AI/StabilityAnimations"
    !mkdir -p $outputs_path
except:
    outputs_path = "."
print(f"Animations will be saved to {outputs_path}")

In [2]:
#@title Connect to the Stability API

import datetime
import os
from pathlib import Path
import shutil
import subprocess
import sys


path = Path('stability-sdk')
if path.exists():
    shutil.rmtree(path)
    !pip uninstall -y stability-sdk

!git clone -b dmarx.anima.dev --recurse-submodules https://github.com/Stability-AI/stability-sdk
!touch ./stability-sdk/src/stability_sdk/interfaces/__init__.py
!pip install ./stability-sdk


from stability_sdk.animation import AnimationArgs, Animator
from stability_sdk.client import generation, generation_grpc # not a huge fan of this but at least it works
from stability_sdk import client
from stability_sdk.utils import (
    color_match_from_string,
    sampler_from_string,
    key_frame_inbetweens,
    key_frame_parse,
    guidance_from_string,
    #curve_to_series,
    image_mix,
    image_to_jpg_bytes,
    image_to_png_bytes,
    image_to_prompt,
    image_xform,
    warp2d_op,
    warp3d_op,
    border_mode_from_str_2d,
    border_mode_from_str_3d,
)

from IPython import display
import panel as pn

pn.extension()


# GRPC endpoint and engines
GRPC_HOST = "" #@param {type:"string"}
API_KEY = "" #@param {type:"string"}
GENERATE_ENGINE_ID = 'stable-diffusion-v1-5'
INPAINT_ENGINE_ID = 'stable-diffusion-v1-5'
TRANSFORM_ENGINE_ID = 'transform-server-v1'


# Connect to Stability API
stub = client.open_channel(GRPC_HOST, api_key=API_KEY)

In [4]:
#@title Settings

args = AnimationArgs()
pn.Row(args.param)

### Prompts

In [5]:
animation_prompts = {
    0: "a painting of a delicious cheeseburger by Tyler Edlin",
    24: "a painting of the the answer to life the universe and everything by Tyler Edlin",
}

negative_prompt = ""
negative_prompt_weight = -1.0


In [6]:
#@title Render the animation

# create folder for frames output
timestring = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
out_dir = os.path.join(outputs_path, timestring)
os.makedirs(out_dir, exist_ok=True)
print(f"Saving animation frames to {out_dir}...")

#args = SimpleNamespace(**Args())
artist = Animator(
    animation_prompts=animation_prompts,
    args=args,
    out_dir=out_dir,
    negative_prompt=negative_prompt,
    negative_prompt_weight=negative_prompt_weight,
    transform_engine_id=TRANSFORM_ENGINE_ID,
    inpaint_engine_id=INPAINT_ENGINE_ID,
    generate_engine_id=GENERATE_ENGINE_ID,
    )
artist.save_settings()
artist.setup_animation()
artist.render_animation(stub=stub)

In [57]:
#@title Create video from frames
skip_video_for_run_all = False #@param {type: 'boolean'}
fps = 12 #@param {type:"number"}

if skip_video_for_run_all == True:
    print('Skipping video creation, uncheck skip_video_for_run_all if you want to run it')
else:
    image_path = os.path.join(out_dir, "frame_%05d.png")
    mp4_path = os.path.join(out_dir, f"{timestring}.mp4")

    print(f"Compiling animation frames to {mp4_path}...")

    cmd = [
        'ffmpeg',
        '-y',
        '-vcodec', 'png',
        '-r', str(fps),
        '-start_number', str(0),
        '-i', image_path,
        '-c:v', 'libx264',
        '-vf',
        f'fps={fps}',
        '-pix_fmt', 'yuv420p',
        '-crf', '17',
        '-preset', 'veryfast',
        mp4_path
    ]
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    if process.returncode != 0:
        print(stderr)
        raise RuntimeError(stderr)

    mp4 = open(mp4_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display.display( display.HTML(f'<video controls loop><source src="{data_url}" type="video/mp4"></video>') )
